In [ ]:
!pip install -q qdrant-client sentence-transformers flashrank

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 353.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently tak

In [ ]:
from tabulate import tabulate
from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
from flashrank import Ranker, RerankRequest
from sentence_transformers import SentenceTransformer

client = QdrantClient(":memory:")

if client.collection_exists("retrieval with reranking"):
    # drop collection if already exist
    client.delete_collection("retrieval with reranking")

client.recreate_collection(
    collection_name="retrieval with reranking",
    vectors_config=models.VectorParams(size=384,
                                       distance=models.Distance.DOT),
)

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
<ipython-input-2-b67cbf417e02>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optiona

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### generate embeddings from text chunks

In [ ]:
# 100 health-related text chunks generated using AI for testing

text_chunks = [
    """Hypertension is a condition in which the blood pressure against the
    artery walls is too high. It is also known as high blood pressure and
    can lead to complications like stroke and heart disease.""",
    """Diabetes is a chronic condition characterized by high blood sugar
    levels. It can be managed through diet, exercise, and medication,
    including insulin.""",
    """Heart disease refers to conditions like coronary artery disease,
    heart attacks, and heart defects. It is often caused by high cholesterol,
    hypertension, smoking, and lack of exercise.""",
    """Asthma is a respiratory condition that causes the airways to narrow
    and swell. It can be triggered by allergens and pollution. Inhalers are
    commonly used for treatment.""",
    """A healthy diet, rich in fruits and vegetables, helps lower the risk
    of heart disease, diabetes, and hypertension. Avoiding processed foods
    and sugars is key.""",
    """Regular exercise strengthens the heart, improves circulation, and
    helps manage weight, reducing the risk of chronic conditions like
    obesity, diabetes, and hypertension.""",
    """Obesity is a risk factor for many diseases, including diabetes, heart
    disease, and arthritis. Maintaining a healthy weight through diet and
    exercise is crucial.""",
    """Smoking is a major cause of lung cancer, heart disease, and
    respiratory conditions. Quitting smoking improves overall health and
    reduces disease risk.""",
    """Mental health is an integral part of overall well-being. Conditions
    like anxiety and depression can lead to physical health problems,
    including heart disease.""",
    """Chronic stress increases the risk of hypertension, heart disease, and
    digestive disorders. Stress management techniques like mindfulness and
    meditation are beneficial.""",
    """Cancer is a group of diseases involving abnormal cell growth.
    Treatment often includes surgery, radiation, chemotherapy, and newer
    treatments like immunotherapy.""",
    """Vaccinations are essential for preventing infectious diseases like
    influenza, measles, and COVID-19. They work by stimulating the body's
    immune response.""",
    """HIV is a virus that attacks the immune system. Early diagnosis and
    antiretroviral therapy are critical in managing the disease and
    preventing the development of AIDS.""",
    """Antibiotics are used to treat bacterial infections. Overuse can lead
    to antibiotic resistance, making some infections harder to treat.""",
    """Allergies occur when the immune system reacts to a foreign substance
    like pollen or pet dander. Symptoms include sneezing, itching, and
    swelling. Antihistamines are commonly used for relief.""",
    """Osteoporosis is a condition that weakens bones, making them more
    likely to break. It is often related to aging and can be prevented
    through calcium-rich diets and weight-bearing exercise.""",
    """Arthritis refers to inflammation of the joints, causing pain and
    stiffness. Treatment often includes anti-inflammatory medications and
    physical therapy.""",
    """A balanced diet that includes whole grains, lean proteins, and
    healthy fats supports brain function and reduces the risk of cognitive
    decline and conditions like dementia.""",
    """Hydration is key for maintaining good health. Drinking enough water
    supports digestion, prevents kidney stones, and helps regulate body
    temperature.""",
    """Sleep is essential for physical and mental health. Lack of sleep is
    linked to conditions like obesity, diabetes, hypertension, and
    depression.""",
    """Vitamin D is important for bone health, as it helps the body absorb
    calcium. It can be obtained through sunlight exposure, foods like fish,
    and supplements.""",
    """High cholesterol levels can lead to plaque buildup in the arteries,
    increasing the risk of heart disease and stroke. Reducing saturated
    fats in the diet can help lower cholesterol.""",
    """Chronic kidney disease occurs when the kidneys lose their ability to
    filter waste from the blood. Managing blood pressure and diabetes helps
    slow disease progression.""",
    """Stroke is caused by interrupted blood flow to the brain, often due to
    blood clots or burst blood vessels. Quick treatment is essential to
    reduce long-term effects.""",
    """Regular screenings for diseases like cancer, diabetes, and
    hypertension help in early detection and treatment, improving outcomes
    and reducing complications.""",
    """Probiotics are beneficial bacteria that help maintain gut health.
    They can be found in fermented foods like yogurt and may improve
    digestion and immune function.""",
    """Chronic obstructive pulmonary disease (COPD) is a lung condition
    caused by long-term exposure to irritants like tobacco smoke. It leads
    to breathing difficulties and requires medical management.""",
    """Depression is a mood disorder characterized by persistent sadness and
    loss of interest. Treatment often includes therapy, medication, and
    lifestyle changes.""",
    """Alzheimer's disease is a progressive brain disorder that leads to
    memory loss and cognitive decline. While there is no cure, medications
    and cognitive therapy can help manage symptoms.""",
    """Parkinson's disease is a neurodegenerative disorder that affects
    movement. It is characterized by tremors, stiffness, and difficulty
    with balance. Medications can help manage symptoms.""",
    """Regular dental care, including brushing and flossing, prevents oral
    health issues like cavities and gum disease, which are linked to broader
    health conditions like heart disease.""",
    """Migraines are severe headaches often accompanied by nausea, vomiting,
    and sensitivity to light. Triggers include stress, certain foods, and
    hormonal changes.""",
    """Anemia occurs when there are not enough healthy red blood cells to
    carry oxygen to the body's tissues. It is often caused by iron
    deficiency and can be treated with supplements.""",
    """Skin cancer is the most common form of cancer. Protecting the skin
    from UV radiation with sunscreen and protective clothing reduces the
    risk of developing skin cancer.""",
    """Chronic pain is long-lasting pain that persists beyond the typical
    recovery period. It is often managed through medications, physical
    therapy, and pain management techniques.""",
    """Food poisoning is caused by consuming contaminated food. Symptoms
    include nausea, vomiting, and diarrhea. Prevention includes proper food
    handling and cooking practices.""",
    """Type 2 diabetes is often linked to obesity and can be managed or even
    prevented with a healthy diet, regular exercise, and maintaining a
    normal weight.""",
    """Epilepsy is a neurological disorder marked by recurrent seizures. It
    is often managed with medications that help control seizure activity.""",
    """Flu is a contagious respiratory illness caused by influenza viruses.
    Annual vaccination is the best way to prevent flu-related
    complications.""",
    """Psoriasis is an autoimmune disease that causes red, scaly patches on
    the skin. It can be managed with topical treatments, light therapy, and
    medications.""",
    """Gout is a form of arthritis caused by the buildup of uric acid in the
    joints, leading to intense pain and swelling. It is often managed
    through diet and medication.""",
    """Anxiety disorders are mental health conditions characterized by
    excessive worry and fear. Treatment often includes therapy, medications,
    and relaxation techniques.""",
    """Chronic fatigue syndrome is a long-term illness marked by extreme
    fatigue that doesn't improve with rest. The cause is unknown, and
    treatment focuses on managing symptoms.""",
    """Liver disease can be caused by factors like alcohol use, hepatitis
    infection, or fatty liver. Early detection and lifestyle changes are
    crucial to prevent liver failure.""",
    """Bipolar disorder is a mental health condition that causes extreme
    mood swings, including emotional highs (mania) and lows (depression). It
    is managed with medications and therapy.""",
    """Glaucoma is an eye condition that damages the optic nerve, often due
    to high eye pressure. Early detection through regular eye exams can help
    preserve vision.""",
    """Multiple sclerosis (MS) is a disease in which the immune system
    attacks the protective covering of nerves. It can cause problems with
    movement, vision, and coordination.""",
    """Polycystic ovary syndrome (PCOS) is a hormonal disorder in women,
    causing irregular menstrual periods and excess male hormone levels. It
    can be managed with lifestyle changes and medication.""",
    """Acne is a skin condition that occurs when hair follicles become
    clogged with oil and dead skin cells. Treatment often includes topical
    creams, medications, and dietary changes.""",
    """Hepatitis is inflammation of the liver, commonly caused by viral
    infections like hepatitis A, B, or C. Vaccination can prevent some forms
    of hepatitis.""",
    """Celiac disease is an immune reaction to eating gluten, a protein found
    in wheat, barley, and rye. A gluten-free diet is necessary to manage
    symptoms.""",
    """Lupus is a chronic autoimmune disease that can affect the skin,
    joints, kidneys, and other organs. It is often treated with
    anti-inflammatory medications and immune suppressants.""",
    """Lyme disease is an infectious disease caused by the bite of a tick
    infected with the bacterium Borrelia burgdorferi. Early treatment with
    antibiotics is crucial.""",
    """Tuberculosis (TB) is a bacterial infection that primarily affects the
    lungs. It is spread through the air and can be treated with a course of
    antibiotics.""",
    """Cholera is an infectious disease caused by eating contaminated food or
    drinking water, leading to severe diarrhea and dehydration. Treatment
    focuses on rehydration and antibiotics.""",
    """Dengue fever is a viral infection transmitted by mosquitoes, causing
    flu-like symptoms, joint pain, and rashes. Preventing mosquito bites is
    key to reducing infection risk.""",
    """Malaria is a life-threatening disease caused by parasites transmitted
    through mosquito bites. Preventative measures include antimalarial
    medication and avoiding mosquito exposure."""
]

# generate embeddings
embeddings = model.encode(text_chunks)

### load embeddings into Qdrant

In [ ]:
for i, embedding in enumerate(embeddings):
    client.upsert(
        collection_name="retrieval with reranking",
        points=[
            PointStruct(
                id=i+1,
                vector=embedding.tolist(),
                payload={"text": text_chunks[i]}
            )
        ]
    )

### Test the Retrieval with reranking feature

In [ ]:
reranker = Ranker()

def retrieve_and_rerank(query):
  """
  This method takes a query as a string, searches and retrieves
  relevant data from the Qdrant database, and then reranks the
  retrieved data based on relevance, printing the scores in a table.
  """
  query_embedded = model.encode(query).tolist()

  search_results = client.search(
      collection_name="retrieval with reranking",
      query_vector=query_embedded,
      limit=5
  )

  # convert Qdrant results into a format suitable for re-ranking
  search_texts = [
    {"text": result.payload['text']} 
    for result in search_results
  ]

  re_ranked_results = reranker.rerank(
    RerankRequest(query,search_texts)
    )
  # Prepare the data for the table
  table_data = [
    (result['score'], result['text']) 
    for result in re_ranked_results
  ]
  # print the table
  print(f"Query: {query}")
  headers = ["Re-ranked Score", "Text"]
  print(tabulate(table_data, headers=headers, tablefmt="grid"))
  print("\n\n")

In [ ]:
# testing out some queries

retrieve_and_rerank("""What are some common methods for
treating and managing arthritis?""")

retrieve_and_rerank("""What are some effective ways to lower
blood pressure and reduce the risk of heart disease?""")

retrieve_and_rerank("""How can I manage diabetes through diet
and lifestyle changes?""")

retrieve_and_rerank("""What lifestyle changes can help prevent
heart disease and improve overall cardiovascular health?""")


Query: What are some common methods for treating and managing arthritis?
+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|   Re-ranked Score | Text                                                                                                                                                                               |
+===================+====================================================================================================================================================================================+
|       0.0742634   | Arthritis refers to inflammation of the joints, causing pain and stiffness. Treatment often includes anti-inflammatory medications and physical therapy.                           |
+-------------------+----------------------------------------------------------------------------------------------